# Deep Weak Stochastic Processes
---

## Meta-Parameters

### Simulation

In [238]:
## Monte-Carlo
N_Euler_Maruyama_Steps = 2
N_Monte_Carlo_Samples = 2

## Grid
N_Grid_Finess = 10**1
Max_Grid = 2

### Quantization
*This hyperparameter describes the proportion of the data used as sample-barycenters.*

In [239]:
Quantization_Proportion = 1

**Note**: Setting *N_Quantizers_to_parameterize* prevents any barycenters and sub-sampling.

# Training Algorithm
---
Given a set of training inputs $\mathbb{X}$ and a stochastic process $(X_t)_{t\geq 0}$ which we can sample from:
1. **For:** x in $\mathbb{X}$:
    - *Simulate:* $\{x\mapsto X_T(\omega_n)\}_{n=1}^N$
    - *Set*: $\hat{\nu}_{x,T}\triangleq \frac1{N}\sum_{n=1}^N \delta_{X_T(\omega_n)}$
2. **Learn:** Wasserstein Barycenters $\hat{\mu}_1,\dots,\hat{\mu}_N
    \in \underset{{\hat{\mu}_n\in\mathscr{P}_{N}(\mathbb{R}^d)}}{\operatorname{argmin}}
    \, \sum_{n=1}^N W_1(\hat{\mu_n},\hat{\nu}_{x,T})$
3. **Train Classifier:** $\hat{f}:x\mapsto \operatorname{n\leq N}\, W_1(\hat{\mu_n},\hat{\nu}_{x,T})$

#### Mode: Code-Testin Parameter(s)

In [240]:
trial_run = True

### Meta-parameters

In [241]:
# Test-size Ratio
test_size_ratio = 1

### Hyperparameters

Only turn of if running code directly here, typically this script should be run be called by other notebooks.  

In [242]:
# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"

### Import

In [243]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Import time separately
import time

Deep Feature Builder - Ready
Deep Classifier - Ready


## Get Internal (Hyper)-Parameter(s)
*Initialize the hyperparameters which are fully-specified by the user-provided hyperparameter(s).*

### Initialize Grid
This is $\mathbb{X}$ and it represents the grid of initial states.

In [244]:
# Get Input Data
x_Grid = np.arange(start=-Max_Grid,
                   stop=Max_Grid,
                   step=(2*Max_Grid/N_Grid_Finess))

# Get Number of Instances in Grid
N_Grid_Instances = len(x_Grid)

# Updater User
print("\u2022 Grid Instances: ", N_Grid_Instances)

• Grid Instances:  10


### Initialize Counting Parameters
Initialize the "conting" type parameters which will help us to determine the length of loops and to intialize object's size later on.  

In [245]:
# Get Internal (Counting) Parameters
N_Quantizers_to_parameterize = round(Quantization_Proportion*N_Grid_Finess)
N_Elements_Per_Cluster = int(round(N_Grid_Instances/N_Quantizers_to_parameterize))

# Update User
print("\u2022",N_Quantizers_to_parameterize," Centers will be produced; from a total datasize of: ",N_Grid_Finess,
      "!  (That's ",Quantization_Proportion,
      " percent).")
print("\u2022 Each Wasserstein-1 Ball should contain: ",
      N_Elements_Per_Cluster, 
      "elements from the training set.")

• 10  Centers will be produced; from a total datasize of:  10 !  (That's  1  percent).
• Each Wasserstein-1 Ball should contain:  1 elements from the training set.


### Set Seed

In [246]:
random.seed(2021)
np.random.seed(2021)
tf.random.set_seed(2021)

---

### Simulate Path
$d X_t = \alpha(t,x)dt + \beta(t,x)dW_t ;\qquad X_0 =x$

### Drift

In [247]:
def alpha(t,x):
    return 0#np.sin(math.pi*t)

### Volatility

In [248]:
def beta(t,x):
    return 1#(t+1)**.5

---

In [249]:
# Initialize List of Barycenters
Wasserstein_Barycenters = []
# Initialize Terminal-Time Empirical Measures
measures_locations_list = []
measures_weights_list = []
# Initialize (Empirical) Weight(s)
measure_weights = np.ones(N_Monte_Carlo_Samples)/N_Monte_Carlo_Samples
# Initialize Quantizer
Init_Quantizer_generic = np.ones(N_Monte_Carlo_Samples)/N_Monte_Carlo_Samples

## Generate $\{\hat{\nu}^{N}_{T,x}\}_{x \in \mathbb{X}}$ Build Wasserstein Cover

#### Get Data

In [250]:
# Update User
print("Current Monte-Carlo Step:")

# Perform Monte-Carlo Data Generation
for i in tqdm(range(N_Grid_Instances)):
    # Get Terminal Distribution Shape
    ###
    # DIRECT SAMPLING
    measures_locations_loop = (np.random.normal(x_Grid[i],np.abs(x_Grid[i]), N_Monte_Carlo_Samples).reshape(-1,))/N_Monte_Carlo_Samples
    
    # Append to List
    measures_locations_list.append(measures_locations_loop.reshape(-1,1))
    measures_weights_list.append(measure_weights)
    
# Update User
print("Done Simulation Step")

100%|██████████| 10/10 [00:00<00:00, 15437.26it/s]

Current Monte-Carlo Step:
Done Simulation Step


#### Get Cover

## Get "Sample Barycenters":
Let $\{\mu_n\}_{n=1}^N\subset\mathcal{P}_1(\mathbb{R}^d)$.  Then, the *sample barycenter* is defined by:
1. $\mathcal{M}^{(0)}\triangleq \left\{\hat{\mu}_n\right\}_{n=1}^N$,
2. For $1\leq n\leq \mbox{N sample barycenters}$: 
    - $
\mu^{\star}\in \underset{\tilde{\mu}\in \mathcal{M}^{(n)}}{\operatorname{argmin}}\, \sum_{n=1}^N \mathcal{W}_1\left(\mu^{\star},\mu_n\right),
$
    - $\mathcal{M}^{(n)}\triangleq \mathcal{M}^{(n-1)} - \{\mu^{\star}\},$
*i.e., the closest generated measure form the random sample to all other elements of the random sample.*

---
**Note:** *We simplify the computational burden of getting the correct classes by putting this right into this next loop.*

---

## Build Dissimilarity (Distance) Matrix
*In this step we build a dissimularity matrix of the dataset on the Wasserstein-1 space.  Namely:*
$$
\operatorname{Mat}_{\# \mathbb{X},\# \mathbb{X}}\left(\mathbb{R}\right)\ni D; \text{ where}\qquad \, D_{i,j}\triangleq \mathcal{W}_1\left(f(x_i),f(x_j)\right)
;
$$
*where $f\in C\left((\mathcal{X},\mathcal{P}_1(\mathcal{Y})\right)$ is the "target" function we are learning.*

**Note**: *Computing the dissimularity matrix is the most costly part of the entire algorithm with a complexity of at-most $\mathcal{O}\left(E_{W} \# \mathbb{X})^2\right)$ where $E_W$ denotes the complexity of a single Wasserstein-1 evaluation between two elements of the dataset.*

In [448]:
# Initialize Disimilarity Matrix
Dissimilarity_matrix_ot = np.zeros([N_Grid_Instances,N_Grid_Instances])


# Update User
print("\U0001F61A"," Begin Building Distance Matrix"," \U0001F61A")
# Build Disimilarity Matrix
for i in tqdm(range(N_Grid_Instances)):
    for j in range(N_Grid_Instances):
        Dissimilarity_matrix_ot[i,j] = ot.emd2_1d(measures_locations_list[j],
                                                  measures_locations_list[i])
# Update User
print("\U0001F600"," Done Building Distance Matrix","\U0001F600","!")

100%|██████████| 10/10 [00:00<00:00, 621.23it/s]

😚  Begin Building Distance Matrix  😚
😀  Done Building Distance Matrix 😀 !


## Initialize Quantities to Loop Over

In [455]:
# Initialize Locations Matrix (Internal to Loop)
measures_locations_list_current = copy.copy(measures_locations_list)
Dissimilarity_matrix_ot_current = copy.copy(Dissimilarity_matrix_ot)
Barycenters_sample = []
Barycenters_indices = np.array([])

# Initialize Classes (In-Sample)
Classifer_Wasserstein_Centers = np.zeros([N_Grid_Instances,N_Quantizers_to_parameterize])

## Get "Sample Barycenters" and Generate Classes

In [456]:
# Update User
print("\U0001F61A"," Begin Identifying Sample Barycenters"," \U0001F61A")

# Identify Sample Barycenters
for i in tqdm(range(N_Quantizers_to_parameterize)):    
    # Distance-Based Sorting
    ## Get Distances from training data
    Distances_Loop = Dissimilarity_matrix_ot_current.sum(axis=1)

    # GET BARYCENTER #
    #----------------#
    ## Identify row with minimum total distance
    Barycenter_index = Distances_Loop.argsort()[:1][0] #_R
    # ## Identify colum with minimum total distance
    # Barycenter_index_C = (Dissimilarity_matrix_ot_current[:,Barycenter_index_R]).argsort()[:1][0]
    ## Get Barycenter
    # ## Update List of Barycenters elements of cluster (indices)
    Barycenters_indices = np.append(Barycenters_indices,Barycenter_index) # HERE!
    measures_locations_list_current[Barycenter_index]

    # GET CLUSTER #
    #-------------#
    # Identify Cluster for this barycenter (which elements are closest to it)
    Cluster_indices = (Dissimilarity_matrix_ot_current[:,Barycenter_index_R]).argsort()[:N_Elements_Per_Cluster]
    # MAKE UPDATES #
    #--------------#
    # ## Get Barycenter 
    Barycenter_loop = measures_locations_list_current[Barycenter_index]
    ## Update Barycenters Array
    if i == 0:
        # Initialize Barycenters Array
        Barycenters_Array = Barycenter_loop
    else:
        # Populate Barycenters Array
        Barycenters_Array = np.append(Barycenters_Array,Barycenter_loop,axis=1)

    ## Update Samples List
    ### Remove from Sample Measures
    for k in range(len(Cluster_indices)):
        index = Cluster_indices[k]
        Dissimilarity_matrix_ot_current[index,:] = math.inf*np.ones(N_Grid_Instances)
        Dissimilarity_matrix_ot_current[index,index] = math.inf

    # Update Classes
    Classifer_Wasserstein_Centers[i,Cluster_indices] = 1

# Update User
print("\U0001F600"," Done Identifying Sample Barycenters","\U0001F600","!")

100%|██████████| 10/10 [00:00<00:00, 6564.88it/s]

😚  Begin Identifying Sample Barycenters  😚
😀  Done Identifying Sample Barycenters 😀 !


In [457]:
pd.DataFrame(Classifer_Wasserstein_Centers)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

### Train Classifier

In this step, we train a deep (feed-forward) classifier:
$$
\hat{f}\triangleq \operatorname{Softmax}_N\circ W_J\circ \sigma \bullet \dots \sigma \bullet W_1,
$$
to identify which barycenter we are closest to.

#### Deep Classifier
Prepare Labels/Classes

In [458]:
# Time-Elapsed Training Deep Classifier
Type_A_timer_Begin = time.time()

Re-Load Grid and Redefine Relevant Input/Output dimensions in dictionary.

#### Train Deep Classifier

In [459]:
# # Re-Load Hyper-parameter Grid
# exec(open('Grid_Enhanced_Network.py').read())
# # Re-Load Helper Function(s)
# exec(open('Helper_Functions.py').read())
%run ParaGAN_Backend.ipynb

# Redefine (Dimension-related) Elements of Grid
param_grid_Deep_Classifier['input_dim'] = [1]
param_grid_Deep_Classifier['output_dim'] = [N_Quantizers_to_parameterize]

Deep Feature Builder - Ready
Deep Classifier - Ready


In [460]:
# Train simple deep classifier
predicted_classes_train, predicted_classes_test, N_params_deep_classifier = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                                                        n_jobs = n_jobs, 
                                                                                                        n_iter =n_iter, 
                                                                                                        param_grid_in=param_grid_Deep_Classifier, 
                                                                                                        X_train = x_Grid, 
                                                                                                        y_train = Classifer_Wasserstein_Centers,
                                                                                                        X_test = x_Grid)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    5.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    5.0s finished


Epoch 1/400
1/1 [==============================] - 0s 1ms/step - loss: 0.3262 - accuracy: 0.1000
Epoch 2/400
1/1 [==============================] - 0s 2ms/step - loss: 0.3228 - accuracy: 0.1000
Epoch 3/400
1/1 [==============================] - 0s 1ms/step - loss: 0.3195 - accuracy: 0.2000
Epoch 4/400
1/1 [==============================] - 0s 1ms/step - loss: 0.3164 - accuracy: 0.2000
Epoch 5/400
1/1 [==============================] - 0s 2ms/step - loss: 0.3134 - accuracy: 0.2000
Epoch 6/400
1/1 [==============================] - 0s 946us/step - loss: 0.3106 - accuracy: 0.2000
Epoch 7/400
1/1 [==============================] - 0s 2ms/step - loss: 0.3078 - accuracy: 0.2000
Epoch 8/400
1/1 [==============================] - 0s 1ms/step - loss: 0.3051 - accuracy: 0.3000
Epoch 9/400
1/1 [==============================] - 0s 1ms/step - loss: 0.3025 - accuracy: 0.3000
Epoch 10/400
1/1 [==============================] - 0s 2ms/step - loss: 0.2999 - accuracy: 0.3000
Epoch 11/400
1/1 [=========

1/1 [==============================] - 0s 2ms/step - loss: 0.1491 - accuracy: 0.8000
Epoch 85/400
1/1 [==============================] - 0s 2ms/step - loss: 0.1468 - accuracy: 0.8000
Epoch 86/400
1/1 [==============================] - 0s 3ms/step - loss: 0.1446 - accuracy: 0.8000
Epoch 87/400
1/1 [==============================] - 0s 2ms/step - loss: 0.1424 - accuracy: 0.8000
Epoch 88/400
1/1 [==============================] - 0s 1ms/step - loss: 0.1402 - accuracy: 0.8000
Epoch 89/400
1/1 [==============================] - 0s 2ms/step - loss: 0.1379 - accuracy: 0.8000
Epoch 90/400
1/1 [==============================] - 0s 3ms/step - loss: 0.1358 - accuracy: 0.8000
Epoch 91/400
1/1 [==============================] - 0s 2ms/step - loss: 0.1336 - accuracy: 0.8000
Epoch 92/400
1/1 [==============================] - 0s 2ms/step - loss: 0.1314 - accuracy: 0.8000
Epoch 93/400
1/1 [==============================] - 0s 2ms/step - loss: 0.1293 - accuracy: 0.8000
Epoch 94/400
1/1 [===============

1/1 [==============================] - 0s 2ms/step - loss: 0.0257 - accuracy: 1.0000
Epoch 168/400
1/1 [==============================] - 0s 4ms/step - loss: 0.0250 - accuracy: 1.0000
Epoch 169/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0244 - accuracy: 1.0000
Epoch 170/400
1/1 [==============================] - 0s 3ms/step - loss: 0.0237 - accuracy: 1.0000
Epoch 171/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0231 - accuracy: 1.0000
Epoch 172/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0224 - accuracy: 1.0000
Epoch 173/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0218 - accuracy: 1.0000
Epoch 174/400
1/1 [==============================] - 0s 1ms/step - loss: 0.0213 - accuracy: 1.0000
Epoch 175/400
1/1 [==============================] - 0s 4ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 176/400
1/1 [==============================] - 0s 1ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 177/400
1/1 [=====

1/1 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 250/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 251/400
1/1 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 252/400
1/1 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 253/400
1/1 [==============================] - 0s 3ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 254/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 255/400
1/1 [==============================] - 0s 4ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 256/400
1/1 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 257/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 258/400
1/1 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 259/400
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 332/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 333/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 334/400
1/1 [==============================] - 0s 3ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 335/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 336/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 337/400
1/1 [==============================] - 0s 6ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 338/400
1/1 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 339/400
1/1 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 340/400
1/1 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 341/400
1/1 [=====

In [461]:
# Time-Elapsed Training Deep Classifier
Type_A_timer_End = time.time() - Type_A_timer_Begin

#### Get Predicted Quantized Distributions

In [484]:
Predictions_Train = np.matmul(Classifer_Wasserstein_Centers,(Barycenters_Array.T))
Predictions_Test = np.matmul(Classifer_Wasserstein_Centers,(Barycenters_Array.T))

In [503]:
hello = 0
# plt.hist(Predictions_Train[hello])
# plt.hist(measures_locations_list[hello].reshape(-1,))
print(np.matmul(predicted_classes_train,(Barycenters_Array.T))[hello])
print(np.matmul(Classifer_Wasserstein_Centers,(Barycenters_Array.T))[hello])
print(measures_locations_list[hello].reshape(-1,))

[ 0.48860907 -0.32398914]
[ 0.48860905 -0.32398913]
[ 0.48860905 -0.32398913]


#### Write Predictions

Compute Performance

In [422]:
# Initialize Wasserstein-1 Error Distribution
W1_errors = np.array([])
Mean_errors = np.array([])

# Get Predicted Means
predicted_means = Predictions_Train.mean(axis=1)
#---------------------------------------------------------------------------------------------#

# Populate Error Distribution
for x_i in range(len(measures_locations_list)):
    # Get Laws
    W1_errors = np.append(W1_errors,ot.emd2_1d(Predictions_Train[x_i,].reshape(-1,),
                                               measures_locations_list[x_i].reshape(-1,)))
    # Get Means
    Mean_errors = np.array(predicted_means[x_i]-np.mean(measures_locations_list[x_i]))
    
#---------------------------------------------------------------------------------------------#
# Compute Error Statistics/Descriptors
W1_Performance = np.array([np.mean(np.abs(W1_errors)),np.mean(W1_errors**2)])
Mean_prediction_Performance = np.array([np.mean(np.abs(Mean_errors)),np.mean(Mean_errors**2)])

Type_A_Prediction = pd.DataFrame({"W1":W1_Performance,"EX":Mean_prediction_Performance},index=["MAE","MSE"])

# Write Performance
Type_A_Prediction.to_latex((results_tables_path+"Type_A_Prediction.tex"))


#---------------------------------------------------------------------------------------------#
# Update User
print(Type_A_Prediction)

           W1        EX
MAE  0.334353  0.304696
MSE  0.420440  0.092840


---

---
# Fin
---

---